In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
import pandas as pd
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from faiss import IndexFlatL2
import pickle
from pathlib import Path
import uuid
import os
import sys

c:\Users\Tegbabu\Intelligent-Complaint-Analysis-FinancialServices\.ienv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sys.path.append(os.path.abspath('../src/'))

In [5]:
from data_process import data_loader

In [6]:
from text_chunker import chunk_narratives

In [7]:
df = data_loader('../data/processed/filtered_complaints.csv')

2025-07-11 01:32:11,241 - INFO - Data loaded successfully from ../data/processed/filtered_complaints.csv


In [8]:
# 1. Text Chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # ~100-150 words, suitable for coherent complaint segments
    chunk_overlap=50,  # Small overlap to maintain context across chunks
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

In [9]:
# Generate chunks
chunks, metadata = chunk_narratives(df, text_splitter)
print(f"Total chunks created: {len(chunks)}")

Total chunks created: 15055


In [12]:
# 2. Generate Embeddings
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = embedding_model.encode(chunks, batch_size=32, show_progress_bar=True)
print(f"Embedding shape: {embeddings.shape}")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Error while downloading from https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...
Error while downloading from https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...


ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b43c3fdb-a85d-457d-8511-34088f78d54e)')